## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [13]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/clairecleveland/Documents/Data Boot Camp/Unit 2/Module 6/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Hilo,19.7297,-155.0900,82.13,74,75,9.22,US,2021-07-30 02:23:25,broken clouds
1,1,Deep River,46.1001,-77.4995,64.15,99,86,4.81,CA,2021-07-30 02:24:10,overcast clouds
2,2,Cape Canaveral,28.4058,-80.6048,77.99,93,1,4.61,US,2021-07-30 02:24:10,clear sky
3,3,Albany,42.6001,-73.9662,68.76,94,92,4.68,US,2021-07-30 02:24:10,overcast clouds
4,4,Georgetown,5.4112,100.3354,82.35,77,20,4.61,MY,2021-07-30 02:21:05,few clouds


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Hilo,19.7297,-155.0900,82.13,74,75,9.22,US,2021-07-30 02:23:25,broken clouds
2,2,Cape Canaveral,28.4058,-80.6048,77.99,93,1,4.61,US,2021-07-30 02:24:10,clear sky
4,4,Georgetown,5.4112,100.3354,82.35,77,20,4.61,MY,2021-07-30 02:21:05,few clouds
11,11,Kupang,-10.1667,123.5833,84.96,57,12,9.82,ID,2021-07-30 02:19:15,few clouds
12,12,Bengkulu,-3.8004,102.2655,81.50,73,37,5.12,ID,2021-07-30 02:24:13,scattered clouds
13,13,Nova Olinda Do Norte,-3.8881,-59.0942,75.13,54,27,2.59,BR,2021-07-30 02:24:14,scattered clouds
19,19,Dhidhdhoo,6.8833,73.1000,82.67,71,82,10.96,MV,2021-07-30 02:24:16,broken clouds
20,20,Puerto Escondido,15.8500,-97.0667,77.83,73,90,6.91,MX,2021-07-30 02:24:16,overcast clouds
22,22,Les Cayes,18.2000,-73.7500,77.83,84,1,5.35,HT,2021-07-30 02:24:17,clear sky
25,25,Ewo,-0.8725,14.8206,77.68,54,52,4.94,CG,2021-07-30 02:24:18,broken clouds


In [16]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                266
City                   266
Lat                    266
Lng                    266
Max Temp               266
Humidity               266
Cloudiness             266
Wind Speed             266
Country                266
Date                   266
Current Description    266
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,82.13,broken clouds,19.7297,-155.0900,
2,Cape Canaveral,US,77.99,clear sky,28.4058,-80.6048,
4,Georgetown,MY,82.35,few clouds,5.4112,100.3354,
11,Kupang,ID,84.96,few clouds,-10.1667,123.5833,
12,Bengkulu,ID,81.50,scattered clouds,-3.8004,102.2655,
13,Nova Olinda Do Norte,BR,75.13,scattered clouds,-3.8881,-59.0942,
19,Dhidhdhoo,MV,82.67,broken clouds,6.8833,73.1000,
20,Puerto Escondido,MX,77.83,overcast clouds,15.8500,-97.0667,
22,Les Cayes,HT,77.83,clear sky,18.2000,-73.7500,
25,Ewo,CG,77.68,broken clouds,-0.8725,14.8206,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 
     
    # Get the latitude and longitude.
    params["location"] = f"{lat},{lng}"
     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd> {Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [36]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))